4. **[20 points] Independent components analysis**

(a) 
\begin{align*}
\frac{\partial \log(|W|)}{\partial W} &= W^{-T} 
\end{align*}
For another part, 
\begin{align*}
\frac{\partial \sum_{i=1}^n \sum_{j=1}^d \log{g'(w^T_j x^{(i)})}}{\partial w^T_l} &= \frac{\partial \sum_{i=1}^n \sum_{j=1}^d -\frac{1}{2}(w^T_j x^{(i)})^2}{\partial w^T_l} \\
&= \sum_{i=1}^n -w^T_l x^{(i)} {x^{(i)}}^T 
\end{align*}
Then take derivative of $l(W)$ and set to 0:
\begin{align*}
& \frac{\partial l(W)}{\partial W} = -WX^TX+mW^{-T} = 0 \\
\Longrightarrow \ & W^TW=m(X^TX)^{-1}
\end{align*}
If $R$ is a rotational matrix and $W'=RW$, we have $W'^TW=(RW)^T(RW)=W^TR^TRW=W^TW=m(X^TX)^{-1}$.
So $W'$ is also a solution, which leads to ambiguity.

(b) 
\begin{align*}
  \frac{\partial \sum_{i=1}^n \sum_{j=1}^d \log{g'(w^T_j x^{(i)})}}{\partial w^T_l} &= \frac{\partial \sum_{i=1}^n \sum_{j=1}^d -|w^T_j x^{(i)}|}{\partial w^T_l} \\
  &= \sum_{i=1}^n -sgn(w^T_l x^{(i)}) {x^{(i)}}^T 
\end{align*}
Then take derivative of $l(W)$
\begin{align*}
  \frac{\partial l(W)}{\partial W} &= mW^{-T}-sgn(WX^T)X
\end{align*}
So 
\begin{align*}
  W:=W+\alpha \left(mW^{-T}-sgn(WX^T)X\right)
\end{align*}

However, in the code, there seems to use a stochastic gradient ascent. So
\begin{align*}
  \frac{\partial l(W)}{\partial W} &= W^{-T}-sgn(Wx^{(i)}){x^{(i)}}^T
\end{align*}
and
\begin{align*}
  W:=W+\alpha \left(W^{-T}-sgn(Wx^{(i)}){x^{(i)}}^T\right)
\end{align*}

(c)

In [1]:
import numpy as np
import scipy.io.wavfile
import os
import numpy as np

def update_W(W, x, learning_rate):
    """
    Perform a gradient ascent update on W using data element x and the provided learning rate.

    This function should return the updated W.

    Use the laplace distribiution in this problem.

    Args:
        W: The W matrix for ICA
        x: A single data element
        learning_rate: The learning rate to use

    Returns:
        The updated W
    """
    
    # *** START CODE HERE ***
    updated_W = W+learning_rate*(np.linalg.inv(W.T)-np.outer(np.sign(W@x),x))
    # *** END CODE HERE ***

    return updated_W


def unmix(X, W):
    """
    Unmix an X matrix according to W using ICA.

    Args:
        X: The data matrix
        W: The W for ICA

    Returns:
        A numpy array S containing the split data
    """

    S = np.zeros(X.shape)


    # *** START CODE HERE ***
    S = X @ W.T
    # *** END CODE HERE ***

    return S


Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

def save_W(W):
    np.savetxt('output/W.txt',W)

def save_sound(audio, name):
    scipy.io.wavfile.write('output/{}.wav'.format(name), Fs, audio)

def unmixer(X):
    M, N = X.shape
    print(X.shape)
    W = np.eye(N)

    anneal = [0.1 , 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01 , 0.01, 0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    for lr in anneal:
        print(lr)
        rand = np.random.permutation(range(M))
        for i in rand:
            x = X[i]
            W = update_W(W, x, lr)

    return W

def main():
    # Seed the randomness of the simulation so this outputs the same thing each time
    np.random.seed(0)
    X = normalize(load_data())

    print(X.shape)

    for i in range(X.shape[1]):
        save_sound(X[:, i], 'mixed_{}'.format(i))

    W = unmixer(X)
    print(W)
    save_W(W)
    S = normalize(unmix(X, W))
    assert S.shape[1] == 5
    for i in range(S.shape[1]):
        if os.path.exists('split_{}'.format(i)):
            os.unlink('split_{}'.format(i))
        save_sound(S[:, i], 'split_{}'.format(i))

if __name__ == '__main__':
    main()

(53442, 5)
(53442, 5)
Separating tracks ...
0.1
0.1
0.1
0.05
0.05
0.05
0.02
0.02
0.01
0.01
0.005
0.005
0.002
0.002
0.001
0.001
[[ 52.83492974  16.79598806  19.9411949  -10.19841036 -20.8977174 ]
 [ -9.9368057   -0.97879563  -4.68186342   8.0430365    1.79099473]
 [  8.31143332  -7.47699382  19.31554724  15.17460858 -14.32640472]
 [-14.66729873 -26.64481368   2.44071692  21.38223128  -8.42094492]
 [ -0.26917605  18.37373974   9.31200636   9.10275731  30.59390495]]
